In [5]:
import torch
from datasets import load_dataset


#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        dataset = load_dataset(path='./ChnSentiCorp', split=split)

        def f(data):
            return len(data['text']) > 30

        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']

        return text


dataset = Dataset('train')

len(dataset), dataset[0]

(9192,
 '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般')

In [12]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('./model_dir/bert-base-chinese')

token

BertTokenizer(name_or_path='./model_dir/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def collate_fn(data):
    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    #把第15个词固定替换为mask
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
print(token.decode(input_ids[0]))
print(token.decode(labels[0]))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape

574
[CLS] 外 观 漂 亮 ， 最 大 的 卖 点 。 键 盘 大 [MASK] 联 想 ｓ１０ 被 淘 汰 的 原 因 。 电 池 [SEP]
，


(torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16, 30]),
 torch.Size([16]))

In [16]:
from transformers import BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"the device is: {device}")

#加载预训练模型
pretrained = BertModel.from_pretrained('./model_dir/bert-base-chinese').to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
token_type_ids = token_type_ids.to(device)
# labels = labels.to(device)
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

out.last_hidden_state.shape

the device is: cuda


torch.Size([16, 30, 768])

In [21]:
#定义下游任务模型
print(f"token vocab_size: {token.vocab_size}")
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.decoder(out.last_hidden_state[:, 15])

        return out


model = Model().to(device)

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

token vocab_size: 21128


torch.Size([16, 21128])

In [24]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print(epoch, i, loss.item(), accuracy)


/home/user_home/xiaomingxu/mambaforge/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0 10.133556365966797 0.0
0 50 8.386566162109375 0.125
0 100 5.440138339996338 0.1875
0 150 5.240077018737793 0.3125
0 200 4.803647041320801 0.3125
0 250 4.083328723907471 0.625
0 300 5.012510299682617 0.375
0 350 4.216627597808838 0.375
0 400 3.810495376586914 0.375
0 450 4.103509426116943 0.4375
0 500 3.2732534408569336 0.625
0 550 3.49190616607666 0.5625
1 0 1.9476391077041626 0.6875
1 50 2.148423910140991 0.625
1 100 2.345219373703003 0.6875
1 150 2.120758533477783 0.75
1 200 2.350722312927246 0.625
1 250 2.3131985664367676 0.5625
1 300 1.674559235572815 0.8125
1 350 2.7869696617126465 0.5
1 400 3.904294013977051 0.3125
1 450 3.5059802532196045 0.4375
1 500 1.6510206460952759 0.6875
1 550 1.0694546699523926 0.875
2 0 0.8782647252082825 0.8125
2 50 1.0888144969940186 0.8125
2 100 0.6853850483894348 0.875
2 150 1.3918399810791016 0.6875
2 200 1.5589661598205566 0.75
2 250 0.5652564764022827 0.8125
2 300 1.232049822807312 0.75
2 350 0.5137938857078552 0.9375
2 400 0.7003031373023987 

In [35]:
# model.save_pretrained("./fine_tune_bert_model")
# token.save_pretrained("./fine_tune_bert_token")
# model.save(model.state_dict(), "model_weights.path")
#torch.save(model.state_dict(), 'model_weights.pth')

model = Model().to(device)
model.load_state_dict(torch.load('model_weights.pth'))
# model = torch.load("model_weights.pth")
# model.to(device)

#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 15:
            break

        print(i)

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

        print(token.decode(input_ids[0]))
        print(token.decode(out[0]), token.decode(labels[0]))

    print(correct / total)


test()

/tmp/ipykernel_1518296/1108353750.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_weights.pth'))


0
[CLS] 该 酒 点 实 在 太 差, 携 程 非 常 不 负 [MASK], 我 花 308 住 豪 华 房, 性 价 比 也 [SEP]
心 责
1
[CLS] 一 直 是 再 重 复 说 男 人 要 得 是 成 绩 [MASK] 美, 女 人 要 得 是 聆 听 观 点 有 点 [SEP]
赞 赞
2
[CLS] 6 月 30 日 入 住 的 。 房 间 总 体 还 行 [MASK] 就 是 有 点 旧 。 周 围 环 境 较 好 。 [SEP]
。 ，
3
[CLS] 一 年 前 我 们 给 孩 子 6 岁 的 儿 子 买 [MASK] 卡 梅 拉 的 第 一 部 ， 孩 子 非 常 喜 [SEP]
了 了
4
[CLS] 非 常 一 般 的 一 本 书 ， 充 满 了 假 想 [MASK] 理 想 主 义 色 彩 ， 建 议 刚 毕 业 的 [SEP]
的 的
5
[CLS] 内 容 还 算 过 的 去 ， 不 过 以 文 采 来 [MASK] ， 就 一 般 。 说 的 只 是 一 些 大 道 [SEP]
说 说
6
[CLS] 位 于 西 环 ， 地 处 香 港 老 城 区 ， 门 [MASK] 有 巴 士 及 电 车 站 ， 交 通 比 较 便 [SEP]
口 口
7
[CLS] 三 个 usb 接 口 居 然 都 在 左 边 ， 接 鼠 [MASK] 很 不 方 便 ， 不 理 解 设 计 师 的 理 [SEP]
盘 标
8
[CLS] 键 盘 太 拥 挤 按 着 不 太 舒 服, 也 容 [MASK] 按 错 键. 不 过 这 体 积 大 概 也 只 [SEP]
易 易
9
[CLS] 我 于 6 月 1 日 再 次 入 住, 住 的 是 [MASK]2 房, 首 先 价 格 由 238 元 涨 到 278 [SEP]
1 1 3 1
10
[CLS] 机 器 外 观 很 不 错 ， 干 净 ， 整 洁 ， [MASK] 感 很 好 。 完 美 屏 ， 音 响 效 果 相 [SEP]
手 手
11
[CLS] 不 错 的 东 西 ， 拿 回 来 ， 第 一 感 觉 [MASK] 是 好 东 西 ！ 包 装 正 规 ， 没 有 拆 [SEP]
就 就
12
[CLS] 这 次 入 住 的 是 大 床 房 ， 房 间 设 施 [MASK] 算 